In [1]:
# 必要なライブラリのインストール
# !pip install transformers torch sentence-transformers
# !pip install openai
!pip install python-dotenv json_repair rouge pipmaster

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.9 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/HKUDS/MiniRAG.git
%cd MiniRAG
!pip install -e .

Cloning into 'MiniRAG'...
remote: Enumerating objects: 500, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 500 (delta 92), reused 68 (delta 58), pack-reused 352 (from 2)
Receiving objects: 100% (500/500), 4.63 MiB | 19.12 MiB/s, done.
Resolving deltas: 100% (261/261), done.
/content/MiniRAG
Obtaining file:///content/MiniRAG
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for minirag-hku (pyproject.toml) ... done
  Created wheel for minirag-hku: filename=minirag_hku-0.0.2-0.editable-py3-none-any.whl size=7106 sha256=89f4d0121a8a2e8abe898cac83a355e6701b85e19cdb5fa697f0a4195cc756c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-2v26lxm7/wheels/68/fd/26/f63d1e2781bd8b8ddd4d165a28bb2d86c7f191d3bb68a100d3
Successfully built minirag-hku


In [8]:
# 必要なライブラリのインポート
import os
import tempfile
from minirag import MiniRAG, QueryParam
from minirag.llm.hf import (
    hf_model_complete,
    hf_embed,
)
from minirag.llm.openai import openrouter_openai_complete
from minirag.utils import EmbeddingFunc
from transformers import AutoModel, AutoTokenizer
import asyncio
import warnings
warnings.filterwarnings('ignore')

In [13]:
# OpenAI APIキーを設定（必要に応じて）
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"

# または、Google Colabのシークレット機能を使用
from google.colab import userdata
try:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("OpenAI APIキーが設定されました")
except:
    print("OpenAI APIキーが見つかりません")


try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    print("Gemini APIキーが設定されました")
except:
    print("Gemini APIキーが見つかりません")


try:
    os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENROUTER_API_KEY')
    print("Openrouter APIキーが設定されました")
except:
    print("Openrouter APIキーが見つかりません")

OpenAI APIキーが見つかりません
Gemini APIキーが設定されました
Openrouter APIキーが設定されました


In [5]:
from google import genai
from google.genai import types
from typing import Union, List, Any, AsyncIterator

async def gemini_complete_if_cache(
    model: str,
    prompt: str,
    system_prompt: Union[str, types.Content] = None,
    history_messages: List[types.Content] = None,
    stream: bool = False,
    **kwargs: Any,
) -> Union[str, AsyncIterator[str]]:
    """
    Google Geminiモデルと対話し、テキストを生成するコア関数。

    Args:
        model (str): 使用するモデル名 (例: 'gemini-2.5-flash-preview-05-20')。
        prompt (str): ユーザーからのプロンプト。
        system_prompt (Union[str, types.Content], optional): システムインストラクション。
        history_messages (List[types.Content], optional): 会話履歴。
        stream (bool, optional): ストリーミングでレスポンスを返すかどうか。デフォルトはFalse。
        **kwargs: `generation_config` や `safety_settings` などの追加パラメータ。

    Returns:
        Union[str, AsyncIterator[str]]:
            - stream=Falseの場合、生成されたテキスト全体 (str)。
            - stream=Trueの場合、テキストのチャンクを返す非同期イテレータ (AsyncIterator[str])。
    """
    client = genai.Client(api_key=GEMINI_API_KEY)

    # contentsを構築
    contents = []
    if history_messages:
        contents.extend(history_messages)
    contents.append(types.Content(role="user", parts=[types.Part(text=prompt)]))

    # GenerateContentConfigで許可されたパラメータのみを抽出
    allowed_params = {
        # 基本パラメータ
        'temperature',
        'top_p',
        'top_k',
        'candidate_count',
        'seed',

        # 出力制御パラメータ
        'max_output_tokens',
        'stop_sequences',
        'response_mime_type',
        'response_schema',

        # ペナルティパラメータ
        'presence_penalty',
        'frequency_penalty',

        # システム設定
        'system_instruction',

        # その他の設定
        'safety_settings',
        'tools',
        'tool_config',
        'automatic_function_calling'
    }

    config_params = {k: v for k, v in kwargs.items() if k in allowed_params}

    if system_prompt:
        if isinstance(system_prompt, str):
            config_params['system_instruction'] = system_prompt
        else:
            # types.Contentオブジェクトの場合、テキスト部分を抽出
            if hasattr(system_prompt, 'parts') and system_prompt.parts:
                config_params['system_instruction'] = system_prompt.parts[0].text
            else:
                config_params['system_instruction'] = str(system_prompt)

    config = types.GenerateContentConfig(**config_params) if config_params else None

    if stream:
        # ストリーミングの場合 - 直接非同期ジェネレータを返す
        return client.aio.models.generate_content_stream(
            model=model,
            contents=contents,
            config=config,
        )
    else:
        # 非ストリーミングの場合
        response = await client.aio.models.generate_content(
            model=model,
            contents=contents,
            config=config,
        )
        try:
            return response.text
        except (ValueError, AttributeError) as e:
            print(f"No valid text content returned. Error: {e}")
            return ""

async def gemini_2_5_flash_complete(
    prompt: str,
    system_prompt: str = None,
    history_messages: List[types.Content] = None,
    stream: bool = False,
    **kwargs: Any,
) -> Union[str, AsyncIterator[str]]:
    """
    'gemini-2.5-flash-preview-'モデルを使用するためのラッパー関数。
    """
    return await gemini_complete_if_cache(
        model="gemini-2.0-flash-001",    # 2.5 はプレビュー版のせいか、結構な頻度で None が返ってくる
        prompt=prompt,
        system_prompt=system_prompt,
        history_messages=history_messages,
        stream=stream,
        **kwargs,
    )

In [6]:
# 埋め込みモデルの設定
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
# LLMの設定
# LLM_MODEL = "Qwen/Qwen3-1.7B"  # または "Qwen/Qwen3-4B", "Qwen/Qwen3-1.7B" など
LLM_MODEL = "jaeyong2/Qwen2.5-3B-Instruct-Ja-SFT"

# 作業ディレクトリの作成
WORKING_DIR = "/tmp/minirag_demo"
os.makedirs(WORKING_DIR, exist_ok=True)

print(f"作業ディレクトリ: {WORKING_DIR}")


# DATA_PATH = args.datapath
# QUERY_PATH = args.querypath
# OUTPUT_PATH = args.outputpath
# print("USING LLM:", LLM_MODEL)
# print("USING WORKING DIR:", WORKING_DIR)

作業ディレクトリ: /tmp/minirag_demo


In [7]:
!pwd

!ls

/content/MiniRAG
assets		    LICENSE		  pyproject.toml  requirements.txt
dataset		    main.py		  README_CN.md	  setup.py
docker-compose.yml  MANIFEST.in		  README_JA.md	  tests
Dockerfile	    minirag		  README.md
graph-visuals	    minirag_hku.egg-info  reproduce


In [9]:
# MiniRAGインスタンスの作成
rag = MiniRAG(
    working_dir=WORKING_DIR,

    # ポスグレ
    # kv_storage="PGKVStorage",
    # vector_storage="PGVectorStorage",
    # graph_storage="PGGraphStorage",

    # llm_model_func=hf_model_complete,
    # llm_model_func=gemini_2_5_flash_complete,
    llm_model_func=openrouter_openai_complete,

    llm_model_max_token_size=200,
    llm_model_name=LLM_MODEL,
    embedding_func=EmbeddingFunc(
        embedding_dim=384,
        max_token_size=1000,
        func=lambda texts: hf_embed(
            texts,
            tokenizer=AutoTokenizer.from_pretrained(EMBEDDING_MODEL),
            embed_model=AutoModel.from_pretrained(EMBEDDING_MODEL),
        ),
    ),
)

print("MiniRAGが初期化されました！")

2025-06-08 22:35:26 - pipmaster.package_manager - INFO - Executing: /usr/bin/python3 -m pip install --upgrade nano-vectordb
2025-06-08 22:35:31 - pipmaster.package_manager - INFO - Command succeeded: /usr/bin/python3 -m pip install --upgrade nano-vectordb


MiniRAGが初期化されました！


In [10]:
#@title ### これをやっておかないと HuggingFace の認証で失敗する

from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForCausalLM
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("HF_TOKENが設定されました")
except:
    print("HF_TOKENが見つかりません")

tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL, token=HF_TOKEN)
model = AutoModelForMaskedLM.from_pretrained(EMBEDDING_MODEL, token=HF_TOKEN)


# tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL, token=HF_TOKEN)
# # モデルの読み込み（メモリ効率を考慮）
# model = AutoModelForCausalLM.from_pretrained(
#     LLM_MODEL,
#     torch_dtype="auto",  # Qwen3では"auto"が推奨
#     device_map="auto",
#     low_cpu_mem_usage=True,
#     token=HF_TOKEN,
# )

HF_TOKENが設定されました


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Some weights of BertForMaskedLM were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# prompt: ロードしたmodelの開放

# メモリ解放のためにモデルを削除
del model
del tokenizer

# PyTorchのキャッシュをクリア (GPUを使用している場合)
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Pythonのガベージコレクションを実行
import gc
gc.collect()

print("モデルとトークナイザーが解放されました。")

モデルとトークナイザーが解放されました。


In [14]:
# 約12分かかった
import time
start_time = time.time()

# サンプルテキストデータ
sample_texts = [
    """
今日は素晴らしい一日でした。朝早く起きて、近所の公園を散歩しました。
桜の花が満開で、とても美しかったです。午後は友人と映画を見に行きました。
「君の名は。」という映画で、とても感動的でした。
夜は家族と一緒に夕食を取り、楽しい時間を過ごしました。
""",
    """
昨日は仕事で大きなプロジェクトが完了しました。
チーム全員で3ヶ月間取り組んできたAIシステムの開発が終わりました。
機械学習モデルの精度が95%を超え、クライアントからも高い評価をいただきました。
今夜はチームメンバーと祝賀会を開く予定です。
""",
    """
週末は料理に挑戦しました。初めてパスタを一から作ってみました。
小麦粉から麺を作るのは思っていたより難しかったですが、
最終的にはとても美味しいカルボナーラができました。
次回はリゾットに挑戦してみたいと思います。
""",
    """
読書が趣味で、最近は村上春樹の「ノルウェイの森」を読んでいます。
主人公の心情描写がとても繊細で、引き込まれます。
また、技術書も読んでおり、「深層学習」について学んでいます。
理論と実践のバランスが取れた良い本だと思います。
"""
]

# データの挿入
print("データを挿入中...")

async def insert_texts(rag_instance, texts):
    for i, text in enumerate(texts):
        print(f"テキスト {i+1}/{len(texts)} を挿入中...")
        await rag_instance.ainsert(text.strip())

    print("すべてのデータが挿入されました！")


# イベントループが既に実行中の場合
try:
    await insert_texts(rag, sample_texts)
except RuntimeError:
    # 新しいループで実行
    asyncio.run(insert_texts(rag, sample_texts))

end_time = time.time()
elapsed_time = end_time - start_time
print(f"処理時間: {elapsed_time:.4f}秒")

データを挿入中...
テキスト 1/4 を挿入中...



Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.32batch/s]


テキスト 2/4 を挿入中...


Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.35batch/s]


Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.70batch/s]


...

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.92batch/s]


テキスト 4/4 を挿入中...


Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.36batch/s]


Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.44batch/s]

すべてのデータが挿入されました！
処理時間: 32.9505秒


In [15]:
# WARNING:minirag:Didn't extract any entities, maybe your LLM is not working
# モデルの精度が足りない？？ どういうときになるのか、LLM に分析してもらったほうがいい
# →日本語SFTモデルに変えたらできたので、シンプルにモデルの性能が足りない。

# あとは
# https://openrouter.ai/deepseek/deepseek-r1-0528:free
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# を使って、openrouter_openai_complete する方法もありそう。



# if not len(all_entities_data):
#     logger.warning("Didn't extract any entities, maybe your LLM is not working")
#     return None
# if not len(all_relationships_data):
#     logger.warning(
#         "Didn't extract any relationships, maybe your LLM is not working"
#     )
#     return None

In [16]:
# 約3分かかった

# # サンプルクエリ
# queries = [
#     "映画について教えて",
#     "仕事のプロジェクトはどうでしたか？",
#     "料理で何を作りましたか？",
#     "読んでいる本について教えて",
#     "散歩について詳しく教えて"
# ]

# # 各モードでクエリを実行。この3つがある
# modes = ["naive", "mini", "light"]

# for query in queries:
#     print(f"\n{'='*50}")
#     print(f"クエリ: {query}")
#     print(f"{'='*50}")

#     for mode in modes:
#         print(f"\n--- {mode.upper()}モード ---")
#         try:
#             answer = rag.query(query, param=QueryParam(mode=mode))     # .replace("\n", "").replace("\r", "")
#             print(f"回答: {answer}")
#         except Exception as e:
#             print(f"エラー: {e}")


async def run_queries(rag_instance, queries):
    # 各モードでクエリを実行。この3つがある
    modes = ["naive", "mini", "light"]

    for query in queries:
        print(f"\n{'='*50}")
        print(f"クエリ: {query}")
        print(f"{'='*50}")

        for mode in modes:
            print(f"\n--- {mode.upper()}モード ---")
            try:
                # 非同期でクエリを実行
                answer = await rag_instance.aquery(query, param=QueryParam(mode=mode))
                print(f"回答: {answer}")
            except Exception as e:
                print(f"エラー: {e}")


start_time = time.time()

sample_queries = [
    "映画について教えて",
    "仕事のプロジェクトはどうでしたか？",
    "料理で何を作りましたか？",
    "読んでいる本について教えて",
    "散歩について詳しく教えて"
]

# イベントループが既に実行中の場合
try:
    await run_queries(rag, sample_queries)
except RuntimeError:
    # 新しいループで実行
    asyncio.run(run_queries(rag, sample_queries))


end_time = time.time()
elapsed_time = end_time - start_time
print(f"処理時間: {elapsed_time:.4f}秒")


クエリ: 映画について教えて

--- NAIVEモード ---
回答: 先日、「君の名は。」という映画を見て、とても感動しました。

--- MINIモード ---
回答: 今週見た映画について、いくつか情報があります。

- **映画**: 物語の語り手が映画を見に行きました。
- **映画**: 見た映画は「君の名は。」という作品で、とても感動的だったと語られています。

--- LIGHTモード ---
回答: 主人公は友人と一緒に映画を見に行き、「君の名は。」という映画を見て感動したようです。

クエリ: 仕事のプロジェクトはどうでしたか？

--- NAIVEモード ---
回答: 昨日、3ヶ月間チームで取り組んできたAIシステムの開発プロジェクトが完了しました。機械学習モデルの精度が95%を超え、クライアントからも高い評価をいただきました。今夜はチームメンバーと祝賀会を開く予定です。


--- MINIモード ---
エラー: 'str' object has no attribute 'get'

--- LIGHTモード ---
回答: ## 仕事のプロジェクトの出来事について

チームは3ヶ月かけてAIシステムを開発し、プロジェクトを完了させました。そのAIシステムの精度は95%を超え、クライアントから高い評価を得ています。プロジェクトの完了を記念して、チームメンバーと祝賀会を開く予定です。


クエリ: 料理で何を作りましたか？

--- NAIVEモード ---
回答: 週末に初めてパスタを一から作り、最終的にはとても美味しいカルボナーラができました。

--- MINIモード ---
エラー: 'str' object has no attribute 'get'

--- LIGHTモード ---
回答: 週末にパスタ作りに挑戦し、カルボナーラを作りました。初めて小麦粉から麺を作ったため、想像以上に難しかったものの、最終的にはとても美味しくできたようです。次はリゾットに挑戦したいと考えているようです。


クエリ: 読んでいる本について教えて

--- NAIVEモード ---
回答: 趣味は読書で、最近は村上春樹の「ノルウェイの森」を読んでおり、主人公の心情描写が非常に繊細で引き込まれるとのことです。
また、技術書も読んでおり、「深

In [17]:
start_time = time.time()

# 日記エントリーのサンプル
sample_diary_entries = [
    {
        "date": "2024-01-15",
        "content": """
今日は新しいプログラミング言語Pythonを学び始めました。
機械学習ライブラリのscikit-learnを使って、初めての分類モデルを作成しました。
データサイエンスの世界は奥が深く、とても興味深いです。
明日はディープラーニングについて学ぶ予定です。
"""
    },
    {
        "date": "2024-01-16",
        "content": """
友人の結婚式に参加しました。とても感動的な式でした。
新郎新婦の幸せそうな笑顔を見て、私も将来について考えさせられました。
式の後のパーティーでは、久しぶりに大学時代の友人たちと再会できました。
みんなそれぞれの道を歩んでいて、刺激を受けました。
"""
    },
    {
        "date": "2024-01-17",
        "content": """
今日は健康診断を受けました。幸い、すべての数値が正常範囲内でした。
医師からは運動を続けるようにアドバイスをもらいました。
最近始めたジョギングの効果が出ているようです。
来月からはジムにも通い始める予定です。
"""
    }
]

# 日記エントリーを挿入
print("日記エントリーを挿入中...")

async def insert_diaries(rag_instance, diary_entries):
    for i, entry in enumerate(diary_entries):
        print(f"日記 {i+1}/{len(diary_entries)} を挿入中...")
        diary_text = f"日付: {entry['date']}\n{entry['content']}"
        await rag_instance.ainsert(diary_text.strip())

    print("\n日記データの挿入が完了しました！")


# イベントループが既に実行中の場合
try:
    await insert_diaries(rag, sample_diary_entries)
except RuntimeError:
    # 新しいループで実行
    asyncio.run(insert_diaries(rag, sample_diary_entries))

end_time = time.time()
elapsed_time = end_time - start_time
print(f"処理時間: {elapsed_time:.4f}秒")

日記エントリーを挿入中...
日記 1/3 を挿入中...


Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.97batch/s]


Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.74s/batch]


日記 2/3 を挿入中...


Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.54batch/s]


Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.02s/batch]


日記 3/3 を挿入中...


Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.92batch/s]


Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.75s/batch]


日記データの挿入が完了しました！
処理時間: 43.7419秒


In [18]:
start_time = time.time()

# 日記に関するクエリ
diary_queries = [
    "プログラミングについて何を学びましたか？",
    "結婚式はどうでしたか？",
    "健康状態はいかがですか？",
    "最近の運動について教えて",
    "友人との再会について詳しく教えて"
]

# イベントループが既に実行中の場合
try:
    await run_queries(rag, diary_queries)
except RuntimeError:
    # 新しいループで実行
    asyncio.run(run_queries(rag, diary_queries))


end_time = time.time()
elapsed_time = end_time - start_time
print(f"処理時間: {elapsed_time:.4f}秒")


クエリ: プログラミングについて何を学びましたか？

--- NAIVEモード ---
回答: 2024年1月15日にPythonを学び始め、機械学習ライブラリのscikit-learnを使って初めての分類モデルを作成しました。データサイエンスの世界は奥深く、興味深いと感じています。また、技術書を読んで「深層学習」について学んでいます。 이론と実践のバランスが取れた良い本だと感じています。明日はディープラーニングについて学ぶ予定です。

--- MINIモード ---
エラー: 'str' object has no attribute 'get'

--- LIGHTモード ---
回答: 週末に、その人はプログラミング言語Pythonを学び始めました。また、機械学習ライブラリのscikit-learnを使って、初めての分類モデルを作成しました。そして、データサイエンスの世界は奥が深く、とても興味深いと感じています。翌日には、ディープラーニングについて学ぶ予定です。


クエリ: 結婚式はどうでしたか？

--- NAIVEモード ---
回答: 2024年1月15日に友人の結婚式に参加し、とても感動的な式だったとのことです。新郎新婦の幸せそうな笑顔を見て、将来について考えさせられたようです。式の後のパーティーでは、久しぶりに大学時代の友人たちと再会し、刺激を受けました。

--- MINIモード ---
エラー: 'str' object has no attribute 'get'

--- LIGHTモード ---
回答: I am sorry, but the provided data does not contain any information about a wedding. Therefore, I cannot answer your question.

クエリ: 健康状態はいかがですか？

--- NAIVEモード ---
回答: 2024年1月17日の健康診断では、すべての数値が正常範囲内であり、医師から運動を続けるようにアドバイスを受けました。最近始めたジョギングの効果が出ているようです。


--- MINIモード ---
エラー: 'str' object has no attribute 'get'

--- LIGHTモード

In [ ]:
Geminiは謎のエラーが出て無理。openrouter 使ったらうまくいった。